In [1]:
import importlib

from amlta.eval.question_generation import process_flows

process_flows = importlib.reload(process_flows)

In [10]:
batch_in_path = process_flows.prepare_batch(n=250, model="gpt-4o-mini")

In [11]:
batch = process_flows.send_batch(batch_in_path)

In [12]:
batch

Batch(id='batch_67ad1a98659081908b5888a968ef1c10', completion_window='24h', created_at=1739397784, endpoint='/v1/chat/completions', input_file_id='file-DkePXe5cBGuBNb8SeW3VqJ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1739484184, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [17]:
batch_out_path = process_flows.retrieve_batch_results(batch.id)

Batch status: in_progress
Batch(id='batch_67ad1a98659081908b5888a968ef1c10', completion_window='24h', created_at=1739397784, endpoint='/v1/chat/completions', input_file_id='file-DkePXe5cBGuBNb8SeW3VqJ', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1739484184, failed_at=None, finalizing_at=None, in_progress_at=1739397786, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=250))
